<a href="https://colab.research.google.com/github/Aunabil4602/Classification-on-COVID-19-Xray-datasets-using-Deep-Learning/blob/master/CV19Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
import tensorflow as tf
from keras.applications.xception import Xception

In [3]:
xcep=Xception(input_shape=[299,299,3],weights='imagenet',include_top=False)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

83689472/83683744 [==============================] - 2s 0us/step


In [0]:
for layer in xcep.layers:
  layer.trainable=False

In [0]:
for layer in xcep.layers:
  print(layer.trainable)

In [0]:
from keras.layers import Flatten,Dense,MaxPooling2D,BatchNormalization,Conv2D,Dropout
from keras.models import Model

In [0]:
xcep.summary()

In [19]:
X=Conv2D(1024,(3,3),strides=(1,1),activation='relu',padding='SAME')(xcep.output)
X=Conv2D(1024,(3,3),strides=(1,1),activation='relu',padding='SAME')(X)
X=BatchNormalization(axis=-1)(X)
X=MaxPooling2D(2,2)(X)
X=Dropout(0.25)(X)
X=Flatten()(X)
X=Dense(512,activation='relu')(X)
pred=Dense(2,activation='softmax')(X)

modelTL=Model(inputs=xcep.input,outputs=pred)
modelTL.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
modelTL.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [20]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amlan107","key":"590556bfeabe7dd9d3f8acc4253a829c"}'}

In [21]:
!pip install kaggle

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [23]:
!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

 99% 1.18G/1.19G [00:10<00:00, 156MB/s]
100% 1.19G/1.19G [00:10<00:00, 121MB/s]


In [0]:
!unzip coronahack-chest-xraydataset.zip

In [0]:
directory='./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/'
fileName='Chest_xray_Corona_Metadata.csv'

In [0]:
import pandas as pd

In [0]:
dataDetails=pd.read_csv('./'+fileName)

In [28]:
print(len(dataDetails))

5910


In [0]:
import os

In [0]:
os.mkdir('./data')
os.mkdir('./data/train')
os.mkdir('./data/test')
os.mkdir('./data/train/normal')
os.mkdir('./data/train/pnemonia')
os.mkdir('./data/test/normal')
os.mkdir('./data/test/pnemonia')

In [0]:
from shutil import copyfile 

In [0]:
total=5910

In [33]:
total_train=0
total_test=0
for it in range(5910):
  if dataDetails['Dataset_type'][it]=='TRAIN':
      total_train +=1
      fname=dataDetails['X_ray_image_name'][it]
      if dataDetails['Label'][it]=='Pnemonia':
        copyfile(directory+'train/'+fname,'./data/train/pnemonia/'+fname)
      else :
        copyfile(directory+'train/'+fname,'./data/train/normal/'+fname)
  else :
    total_test +=1
    fname=dataDetails['X_ray_image_name'][it]
    if dataDetails['Label'][it]=='Pnemonia':
      copyfile(directory+'test/'+fname,'./data/test/pnemonia/'+fname)
    else :
      copyfile(directory+'test/'+fname,'./data/test/normal/'+fname)

print(total_test)
print(total_train)

624
5286


In [34]:
print(total_train+total_test)

5910


In [0]:
from os import listdir

In [0]:
fileList1=listdir('./data/train/pnemonia/')
fileList2=listdir('./data/test/pnemonia/')
fileList3=listdir('./data/test/normal/')
fileList4=listdir('./data/train/normal/')

In [37]:
print(len(fileList1)+len(fileList2)+len(fileList3)+len(fileList4))

5910


In [0]:
##### generators
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator 

In [0]:
dataGen=ImageDataGenerator(rescale=1./255,rotation_range=5.0,zoom_range=[0.95,1.05],width_shift_range=0.05,height_shift_range=0.05,fill_mode='constant',cval=255)

In [0]:
dataGenOrigin=ImageDataGenerator(rescale=1./255)

In [41]:
trainSet=dataGen.flow_from_directory('./data/train',target_size=(299,299),batch_size=32,class_mode='categorical',color_mode='rgb')
testSet=dataGen.flow_from_directory('./data/test',target_size=(299,299),batch_size=32,class_mode='categorical',color_mode='rgb')

Found 5286 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [49]:
testSetOrigin=dataGenOrigin.flow_from_directory('./data/test',target_size=(299,299),batch_size=32,class_mode='categorical',color_mode='rgb')

Found 624 images belonging to 2 classes.


In [0]:

from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.5,patience=3, min_lr=0.00001,verbose=1)

In [43]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=1,verbose=1)


Epoch 1/1
166/166 [==============================] - 222s 1s/step - loss: 0.7783 - acc: 0.8816 - val_loss: 0.5790 - val_acc: 0.6779


In [45]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=10,verbose=1,callbacks=[reduce_lr])

Epoch 1/10
166/166 [==============================] - 217s 1s/step - loss: 0.2203 - acc: 0.9245 - val_loss: 1.5134 - val_acc: 0.6282
Epoch 2/10
166/166 [==============================] - 207s 1s/step - loss: 0.1478 - acc: 0.9438 - val_loss: 2.1632 - val_acc: 0.6298
Epoch 3/10
166/166 [==============================] - 208s 1s/step - loss: 0.1449 - acc: 0.9465 - val_loss: 1.0718 - val_acc: 0.6474
Epoch 4/10
166/166 [==============================] - 208s 1s/step - loss: 0.1223 - acc: 0.9576 - val_loss: 1.5935 - val_acc: 0.6394
Epoch 5/10
166/166 [==============================] - 208s 1s/step - loss: 0.2007 - acc: 0.9455 - val_loss: 1.0989 - val_acc: 0.6298
Epoch 6/10
166/166 [==============================] - 207s 1s/step - loss: 0.1274 - acc: 0.9516 - val_loss: 2.7911 - val_acc: 0.6250
Epoch 7/10
166/166 [==============================] - 208s 1s/step - loss: 0.1099 - acc: 0.9586 - val_loss: 1.4866 - val_acc: 0.6619
Epoch 8/10
166/166 [==============================] - 209s 1s/step - 

In [52]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=10,verbose=1,callbacks=[reduce_lr])

Epoch 1/10
166/166 [==============================] - 218s 1s/step - loss: 0.0900 - acc: 0.9665 - val_loss: 0.7468 - val_acc: 0.6715
Epoch 2/10
166/166 [==============================] - 208s 1s/step - loss: 0.0877 - acc: 0.9661 - val_loss: 1.7213 - val_acc: 0.6266
Epoch 3/10
166/166 [==============================] - 209s 1s/step - loss: 0.0984 - acc: 0.9646 - val_loss: 2.3920 - val_acc: 0.6282
Epoch 4/10
166/166 [==============================] - 209s 1s/step - loss: 0.0979 - acc: 0.9641 - val_loss: 1.2575 - val_acc: 0.6362
Epoch 5/10
166/166 [==============================] - 208s 1s/step - loss: 0.0853 - acc: 0.9629 - val_loss: 2.7544 - val_acc: 0.6442
Epoch 6/10
166/166 [==============================] - 210s 1s/step - loss: 0.0858 - acc: 0.9675 - val_loss: 0.8619 - val_acc: 0.6314
Epoch 7/10
166/166 [==============================] - 208s 1s/step - loss: 0.0798 - acc: 0.9701 - val_loss: 0.8234 - val_acc: 0.6298
Epoch 8/10
166/166 [==============================] - 209s 1s/step - 

In [55]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=10,verbose=1,callbacks=[reduce_lr])

Epoch 1/10
166/166 [==============================] - 214s 1s/step - loss: 0.0702 - acc: 0.9752 - val_loss: 1.3274 - val_acc: 0.6394
Epoch 2/10
166/166 [==============================] - 205s 1s/step - loss: 0.0661 - acc: 0.9765 - val_loss: 0.2892 - val_acc: 0.6442
Epoch 3/10
166/166 [==============================] - 204s 1s/step - loss: 0.0586 - acc: 0.9781 - val_loss: 1.4898 - val_acc: 0.6266
Epoch 4/10
166/166 [==============================] - 201s 1s/step - loss: 0.0633 - acc: 0.9750 - val_loss: 2.4231 - val_acc: 0.6298
Epoch 5/10
166/166 [==============================] - 204s 1s/step - loss: 0.0748 - acc: 0.9748 - val_loss: 0.7703 - val_acc: 0.7468
Epoch 6/10
166/166 [==============================] - 204s 1s/step - loss: 0.0679 - acc: 0.9760 - val_loss: 0.7095 - val_acc: 0.6827

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 7/10
166/166 [==============================] - 204s 1s/step - loss: 0.0449 - acc: 0.9843 - val_loss: 1.3964 - val_

In [47]:
e1=modelTL.evaluate_generator(testSetOrigin,verbose=1)

166/166 [==============================] - 116s 700ms/step


In [48]:
print(e1)

[1.3349014520645142, 0.7572833895683289]


In [50]:
e1=modelTL.evaluate_generator(testSetOrigin,verbose=1)

20/20 [==============================] - 14s 705ms/step


In [51]:
print(e1)

[2.1292190551757812, 0.6282051205635071]


In [56]:
e1=modelTL.evaluate_generator(testSetOrigin,verbose=1)
print(e1)

20/20 [==============================] - 14s 700ms/step
[0.5858509540557861, 0.6875]
